In [1]:
import pandas as pd
import numpy as np

import unicodedata

years = [1993,1997,2002,2007,2012]

In [2]:
liste_df = {}

#import de chq csv
for year in years:
    key = year
    liste_df[year] = pd.read_csv('./attribution_groupes/legis_raw/%d.csv' % year)

In [3]:
liste_df[1993].head()

,Unnamed: 0,Candidat,Circ,Departement,Parti,Score
0,0,Jacques Boyon,1.0,-circonscription-ain,RPR,"46,47%"
1,1,Pierre Fromont,1.0,-circonscription-ain,Divers gauche,"24,11%"
2,2,Bernard Aulagne,1.0,-circonscription-ain,FN,"10,28%"
3,3,Marc de Antoni,1.0,-circonscription-ain,Les Verts,"9,86%"
4,4,Lionel Mornet,1.0,-circonscription-ain,PCF,"3,98%"


In [4]:
#formater les deps

for year in years:
    liste_df[year]['Departement'] = liste_df[year]['Departement'].apply(lambda x: x[17:])

In [5]:
liste_df[1993].head()

,Unnamed: 0,Candidat,Circ,Departement,Parti,Score
0,0,Jacques Boyon,1.0,ain,RPR,"46,47%"
1,1,Pierre Fromont,1.0,ain,Divers gauche,"24,11%"
2,2,Bernard Aulagne,1.0,ain,FN,"10,28%"
3,3,Marc de Antoni,1.0,ain,Les Verts,"9,86%"
4,4,Lionel Mornet,1.0,ain,PCF,"3,98%"


In [11]:
#correction d'une ligne où parti et candidat ont été fusionné
liste_df[2002][liste_df[2002].Float == True]

,Unnamed: 0,Candidat,Circ,Departement,Parti,Score,Float
823,823,Jean-MichelDivers droite Segantini,11.0,bouches-du-rhone,NaN,"2,11%",True


In [12]:
#pas de correction au niveau du candidat car on utilisera uniquement le parti pour former les groupes
liste_df[2002]['Parti'] = liste_df[2002]['Parti'].replace(np.nan,'Divers droite',regex=True)

#formater certains partis affichés avec des sauts de lignes / tabs.
for year in years:
        liste_df[year]['Parti'] = liste_df[year]['Parti'].apply(lambda x: x.replace('\n\t\t\t\t\t\t\t\t\t',''))

In [13]:
#Création d'une df vide pour concaténer
df_full = pd.DataFrame(columns = ('year','Departement', 'Circ', 'Parti', 'Score'))

In [14]:
df_full

,year,Departement,Circ,Parti,Score


In [15]:
#append chq df
for year in years:
    df_full = df_full.append(pd.DataFrame({'year': year,
                             'Departement':liste_df[year].Departement,
                            'Circ' : liste_df[year].Circ,
                            'Parti' : liste_df[year].Parti,
                            'Score' :liste_df[year].Score}))

In [16]:
df_full.head()

,Circ,Departement,Parti,Score,year
0,1.0,ain,RPR,"46,47%",1993.0
1,1.0,ain,Divers gauche,"24,11%",1993.0
2,1.0,ain,FN,"10,28%",1993.0
3,1.0,ain,Les Verts,"9,86%",1993.0
4,1.0,ain,PCF,"3,98%",1993.0


In [17]:
#reste des tabs dans les partis

df_full.Parti = df_full.Parti.apply(lambda x: x.replace('\t\t\t\t\t\t\t\t\t',''))

In [19]:
#lowercase les partis

df_full.Parti = df_full.Parti.apply(lambda x: x.lower())

In [25]:
#se débarasser des accents

def rem_accents(s):
    s2 = s.decode('utf-8')
    nfkd_form = unicodedata.normalize('NFKD', s2)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii

In [26]:
#test
rem_accents(df_full[df_full.Parti == 'les nouveaux écologistes'].Parti[5])

'les nouveaux ecologistes'

In [27]:
#application
df_full.Parti = df_full.Parti.apply(rem_accents)

In [28]:
#on aura moins de doublons ainsi.
df_full.head(20)

,Circ,Departement,Parti,Score,year
0,1.0,ain,rpr,"46,47%",1993.0
1,1.0,ain,divers gauche,"24,11%",1993.0
2,1.0,ain,fn,"10,28%",1993.0
3,1.0,ain,les verts,"9,86%",1993.0
4,1.0,ain,pcf,"3,98%",1993.0
5,1.0,ain,les nouveaux ecologistes,"2,55%",1993.0
6,1.0,ain,lo,"1,54%",1993.0
7,1.0,ain,lcr,"1,22%",1993.0
8,1.0,ain,divers,"0,00%",1993.0
9,2.0,ain,rpr,"46,53%",1993.0


In [29]:
#Arranger le format des scores

df_full.Score = df_full.Score.apply(lambda x: float(x.replace(',', '.').replace('%', '')))

In [30]:
df_full.head()

,Circ,Departement,Parti,Score,year
0,1.0,ain,rpr,46.47,1993.0
1,1.0,ain,divers gauche,24.11,1993.0
2,1.0,ain,fn,10.28,1993.0
3,1.0,ain,les verts,9.86,1993.0
4,1.0,ain,pcf,3.98,1993.0


In [31]:
#Une df pour voir qui a un bon score moyen et savoir qui mettre dans 'autres'

parties = pd.DataFrame(columns=('Parti', 'Score_moyen'))
for parti in df_full.Parti.unique():
    score_m = float(df_full[df_full.Parti == parti].Score.mean())
    table = np.array([[parti, score_m]])
    parties = parties.append(pd.DataFrame(table, columns=['Parti', 'Score_moyen']), ignore_index=True)

In [32]:
#passer par un np array a modifié le type de cette colonne

parties.Score_moyen = parties.Score_moyen.apply(float)

In [33]:
#Au fond on veut une df sortée

parties = parties.sort_values(by='Score_moyen')

In [34]:
len(parties)

420

In [29]:
#Ces 3 cells permettent de calculer des taux de ressemblance des strings


#imp = ['UMP', 'Parti socialiste', 'RPR', 'PS', 'UDF', 'rpr', 'Parti communiste', 'PCR' ,'ps','pc','Union pour un mouvement populaire', 'Divers droite', 'UMP', 'FN','Front National']

In [30]:
#from difflib import SequenceMatcher
#def similar(a, b):
#    return SequenceMatcher(None, a, b).ratio()

In [31]:
#for partipol in imp:
#    parties[partipol] = parties.Parti.apply(lambda x: similar(x, partipol))

In [36]:
#possible qu'on range les partis ayant un score moyen inférieur à 2% dans autres.
parties[-200:]

,Parti,Score_moyen
285,parti chretien democrate,1.330921
160,alternative nationale,1.340000
375,mouvement affirmation,1.350000
327,front de droite,1.360000
72,div,1.372581
388,mpr,1.380000
223,union elargie des seniors,1.380000
35,mouvement des democrates,1.398800
257,te'aevei'a,1.400000
381,collectif des inkoruptibles,1.430000


In [33]:
parties[:460]

,Parti,Score_moyen,UMP,Parti socialiste,RPR,PS,UDF,rpr,Parti communiste,PCR,ps,pc,Union pour un mouvement populaire,Divers droite,FN,Front National
205,Rassemblement France Réunion,0.000000,0.000000,0.177778,0.125000,0.000000,0.062500,0.062500,0.266667,0.062500,0.064516,0.064516,0.290323,0.095238,0.064516,0.325581
387,Mouvement EBRT,0.000000,0.117647,0.133333,0.117647,0.000000,0.000000,0.000000,0.200000,0.117647,0.000000,0.000000,0.382979,0.222222,0.000000,0.285714
185,Parti des musulmans de France,0.000000,0.062500,0.311111,0.062500,0.064516,0.062500,0.125000,0.444444,0.062500,0.064516,0.064516,0.290323,0.238095,0.064516,0.186047
372,Nouveaux entrepreneurs de l?espace rural français,0.000000,0.000000,0.181818,0.000000,0.000000,0.000000,0.113208,0.151515,0.000000,0.076923,0.076923,0.216867,0.222222,0.038462,0.187500
366,Appel de Béziers,0.000000,0.000000,0.060606,0.000000,0.000000,0.000000,0.200000,0.060606,0.000000,0.210526,0.105263,0.120000,0.266667,0.000000,0.064516
400,Parti universaliste de France,0.000000,0.060606,0.565217,0.060606,0.062500,0.060606,0.121212,0.521739,0.060606,0.062500,0.062500,0.253968,0.372093,0.062500,0.272727
322,L'engagement Pacifique et son Ethique,0.000000,0.048780,0.296296,0.048780,0.050000,0.000000,0.000000,0.185185,0.048780,0.050000,0.050000,0.281690,0.078431,0.000000,0.269231
249,Démocratie et Légitimité,0.000000,0.000000,0.232558,0.000000,0.000000,0.066667,0.066667,0.186047,0.000000,0.000000,0.068966,0.233333,0.200000,0.000000,0.195122
382,Mouvement anti-radars,0.000000,0.076923,0.256410,0.000000,0.000000,0.000000,0.153846,0.205128,0.000000,0.080000,0.000000,0.464286,0.277778,0.000000,0.432432
132,Union droite républicaine,0.000000,0.068966,0.238095,0.000000,0.000000,0.068966,0.137931,0.190476,0.000000,0.071429,0.142857,0.542373,0.410256,0.000000,0.250000


In [78]:
#Ce gros dico a été construit avec Wikipedia pour décrire quels partis sont contenus par chaque groupe politique.
#Je vais le repasser à la moulinette de lower et rem_accents histoire qu'on ne garde que les libellés qui comptent.
#Il ne permettra pas de classer tous les partis, mais à la fin on mettra tous les partis de score moyen en dessosu d'un certain
#niveau dans autres.

d = { 'EG' : ['Part des travailleurs','Parti des travailleurs','Génération écologie',
              'Les nouveaux écologistes','Les Verts','Bâtir le pays Martinque','Batir le pays Martinique',
              'Mouvement des radicaux de gauche','PRG','Parti radical de gauche','RDM','PCF','Combat ouvrier','Ecologiste',
              "Cap sur l'avenir",'Cap sur l','Parti progressiste martiniquais','Tavini Huiraatira',
              'Parti progressiste et démocratique guadeloupéen','PCR','Parti progressiste démocratique guadeloupéen',
              'Conseil national des comités populaires','Front de Libération Nationale Kanak et Socialiste','front de gauche',
              'Front de gauche','FLNKS','MEI','RCF','LO','LCR','EELV','EXG','FASE','Parti communiste martiniquais','MUP',
              'Parti humaniste','Parti Humaniste','Parti Social Mahorais','Parti occitan','Partit occitan',
              'Parti communiste réunionnais','Parti Communiste réunionnais','Parti communiste guadeloupéen','Communistes',
              'Communiste Réunionnais','Parti Communiste Guadeloupéen', 'Parti Communiste Réunionnais',
              'Front de libération nationale kanak et socialiste', 'Bâtir le pays Martinique', 'communistes en somme',
             'faire front a gauche', 'eologiste', 'solidarite, ecologie, gauche alternative', 'ecologie citoyenne',
             'convention pour une alternative progressiste'] ,
    'G': ['écologie, gauche alternative','Gauche moderne','Parti socialiste guyannais','Divers gauche','Mouvement Bretagne Progrès',
          'PS dissident','Nouvelle gauche','La Nouvelle Gauche','Unionn démocratique bretonne','Walwari','Réunion espoir',
          'Corse social-démocrate','Mouvement pour la Réunion','Mouvement populaire franciscains','MRC','UDB','MAJ','DVG',
          'MRC','SOC','PS','Parti radical socialiste', 'Parti socialiste','Parti socialiste guyanais','Parti Socialiste Guyanais',
          'Gauche socialiste','Parti Socialiste Polynésien','Parti Socialiste Réunionnais','Parti Socialiste Guadeloupéen',
          'Priorité socialiste Réunion','Renouveau Socialiste', 'corse social democrate'],
    'C': ['Le Centre pour la France','Le Nouveau Centre','Nouveau centre','UDF dissident',
          'MoDem','Calédonie ensemble','Union Locale Démocrate','Union pour la Démocratie','Parti pour la population',
          'MDM','Partie fédéraliste','Parti fédéraliste','Partie fédéraliste',
          'Parti radical Valoisien','Parti radical valoisien','UDF', 'alliance centriste', 'mouvement des reformateurs'],
    'D': ['Divers droite','Objectif Guadeloupe','Entente républicaine','Camp liberal corsiste','Avenir Ensemble',
           'Démocratie libérale','Union niçoise','RPR dissident','Osons oser','Divers drotie','Forces martiniquaises de progrès',
           'Te niu hau manahune','Droite républicaine centriste','Mouvement libéral martiniquais','rpr','RPR',
           'Tahoeraa huiraatira','Union pour un mouvement populaire','LDI-CNIP','LDI-MPF','Rassemblement pour la Calédonie',
           'Union pour la France','Nouvelle droite populaire','Union républicaine populaire','Union républicaine populaire',
           'UMP','CNI','DVD', "rassemblement des francais de l'etranger", 'cpnt'],
    'ED': ['Debout la République', 'Union du peuple alsacien', 'Ligue savoisienne','Extrême droite',
           'Rassemblement du peuple franc-comtois','Rassemblement bleu Marine','Unser Land',"Alsace d'Abord",'Ligue du Sud',
           'Unione per una Suluzione Pulitica','Parti National Radical','Unione Nazionale','FN','EXD','MPF','Ligue nationaliste',
          'mnr.png'],
    'Autres': ['Corsica naziune','Mouvement Citterien', 'Mouvement citerrien', 'Nouvelle donne','Mouvement Citerrien',
               'Droit de chasse', 'Droit  de Chasse','Esquerra Republicana de Catalunya', 'Unitat Catalana',
               'Parti nationaliste basque','Union du peuple corse','sans étiquette',"Sans étiquette",
               'Mouvement indépendantiste martiniquais','Martinique nouvelle','Alliance des Français pour le progrès',
               'Palima','Cornouaille 2010','Femu a Corsica','Parti régionaliste martiniquais','Euskal herria bai','UPLD',
               'M','MDES','MHAN','REG','DIV','RNB','US4J','Parti pirate','MAP','map','rnb','AMPR','MPR',
               'Parti nationaliste basque','Parti anti sioniste','Parti du plaisir','Anticolonialistes',
              'Front de Libération de la Polynésie', 'Free dom', 'Union pour la Démocratie', 'RRR', "solidarité et progrès",
              'union pour la democratie ', 'upld-tavini', "mouvement de decolonisation et d'emancipation sociale",
              'nouvel elan pour mayotte', 'o porinetia to tatou ai', 'rautahi', "mouvement pour l'autodetermination",
              'corsica libera']
    }

In [79]:
#transformation du dico avec lower et rem_accents
d2 = {}

for key in d.keys():
    l = []
    for parti in d[key]:
        l.append(rem_accents(parti.lower()))
    d2[key] = list(set(l))

In [80]:
d2['Autres']

['upld',
 'anticolonialistes',
 'corsica naziune',
 'parti anti sioniste',
 'parti pirate',
 'parti regionaliste martiniquais',
 'cornouaille 2010',
 'unitat catalana',
 'alliance des francais pour le progres',
 'solidarite et progres',
 'femu a corsica',
 'mouvement citterien',
 'parti nationaliste basque',
 'droit de chasse',
 'upld-tavini',
 'corsica libera',
 'o porinetia to tatou ai',
 'mhan',
 'sans etiquette',
 'rrr',
 'front de liberation de la polynesie',
 'mouvement independantiste martiniquais',
 'mouvement citerrien',
 "mouvement de decolonisation et d'emancipation sociale",
 'free dom',
 'reg',
 'us4j',
 'union du peuple corse',
 'map',
 'union pour la democratie',
 'rnb',
 'droit  de chasse',
 'union pour la democratie ',
 'mdes',
 'palima',
 "mouvement pour l'autodetermination",
 'parti du plaisir',
 'esquerra republicana de catalunya',
 'rautahi',
 'mpr',
 'euskal herria bai',
 'm',
 'nouvel elan pour mayotte',
 'martinique nouvelle',
 'ampr',
 'nouvelle donne',
 'div']

In [81]:
#fct qui transforme un parti en son groupe politique
def grouping(party):
    in_groups = []
    
    for group in d2.keys():
        if party in d2[group]:
            in_groups.append(group)
        else:
            pass
        
    if len(in_groups) == 0:
#        print '%s not in dict !' % party
        return party
    elif len(in_groups) > 1:
        print ('%s in groups :' % party), in_groups
        return party
    else:
        return in_groups[0]

In [82]:
#test
test = df_full.Parti.apply(grouping).unique()

In [83]:
#vérifier qu'aucun des partis restants n'apparait dans notre dict
for party in test:
    for group in d.keys():
        if party in d[group]:
            print party + ':' + d[group]
            
#Si ça n'imprime rien c'est buen

In [84]:
#Voir un peu qui il reste de non trié avec un score > 2
parties[(parties.Parti.isin(test)) & (parties.Score_moyen > 2)]

,Parti,Score_moyen
216,parti d'en rire,2.130000
141,convention citoyenne,2.266667
42,union democratique bretonne,2.320000
311,breizhistance,2.320000
54,mouvement region savoie,2.340000
147,u partitu di a corsica,2.350000
182,union pour la liberation de la guadeloupe,2.403333
201,nissa rebela,2.412500
406,te ao hou o te fenua,2.600000
236,nofwap,2.660000


In [87]:
#Pour l'instant on estimera que le travail fait suffit.
#C'est parti pour le grand grouping

#ptite fonction pour classer tout ce qui n'a pas été classé en autres et laisser les groupes politiques intacts
def autriser(parti):
    if parti in d2.keys():
        return parti
    else:
        return 'Autres'
    
#test
autriser('parti machin')

'Autres'

In [92]:
#c'est parti
partis_groupes = df_full.Parti.apply(grouping).apply(autriser)

In [93]:
df_full.Parti = partis_groupes

In [94]:
df_full.head()

,Circ,Departement,Parti,Score,year
0,1.0,ain,D,46.47,1993.0
1,1.0,ain,G,24.11,1993.0
2,1.0,ain,ED,10.28,1993.0
3,1.0,ain,EG,9.86,1993.0
4,1.0,ain,EG,3.98,1993.0


In [ ]:
df_full.Parti.unique()

In [98]:
#test contre le csv d'origine
df_full[(df_full.year == 2012.0) & (df_full.Departement == 'indre-et-loire') & (df_full.Circ == 4.0)]

,Circ,Departement,Parti,Score,year
1969,4.0,indre-et-loire,G,39.65,2012.0
1970,4.0,indre-et-loire,D,35.93,2012.0
1971,4.0,indre-et-loire,ED,11.40,2012.0
1972,4.0,indre-et-loire,EG,4.78,2012.0
1973,4.0,indre-et-loire,EG,2.52,2012.0
1974,4.0,indre-et-loire,C,2.12,2012.0
1975,4.0,indre-et-loire,Autres,0.88,2012.0
1976,4.0,indre-et-loire,ED,0.74,2012.0
1977,4.0,indre-et-loire,Autres,0.65,2012.0
1978,4.0,indre-et-loire,Autres,0.43,2012.0


In [100]:
#OK pour finir grouper et sommer
df2 = df_full.groupby(['year', 'Departement', 'Circ', 'Parti']).sum()

In [104]:
#test contre le csv d'origine
df2.loc[2012.0, 'indre-et-loire', 4.0,:]

Score
year   Departement    Circ Parti        
2012.0 indre-et-loire 4.0  Autres   2.86
                           C        2.12
                           D       35.93
                           ED      12.14
                           EG       7.30
                           G       39.65

In [105]:
#save
outpath = './legis_clean/legis_clean.csv'
df2.to_csv(outpath, encoding='utf-8')